## 0. 텐서플로우 메모리 관리

In [2]:
# 워닝 무시
import warnings
warnings.filterwarnings('ignore')

In [3]:
import tensorflow as tf


gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
        print(e)

2022-08-01 04:46:10.957479: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-01 04:46:11.052811: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-01 04:46:11.054856: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


## 1. 패키지 로드

In [40]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.losses import MeanSquaredLogarithmicError
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

## 2. 데이터 로드

In [4]:
train_df = pd.read_csv('./data/train.csv') # 학습용 데이터
test_df = pd.read_csv('./data/test.csv') # 테스트용 데이터
val_df = pd.read_csv('./data/val.csv') # 검증용 데이터

In [6]:
X_features = train_df.drop(columns=['ID'])

In [7]:
X_val_features = val_df.drop(columns=['ID', 'Class'])
y_val_features = val_df['Class']

In [8]:
X_test_features = test_df.drop(columns=['ID'])

## 3. 데이터 전처리

In [36]:
min_max_scaler = MinMaxScaler(feature_range=(0, 1))
x_train_scaled = min_max_scaler.fit_transform(X_features)
x_test_scaled = min_max_scaler.transform(X_test_features)
x_val_scaled = min_max_scaler.transform(X_val_features)

In [37]:
type(x_val_scaled)

numpy.ndarray

## 4. 모델 생성

In [10]:
class AutoEncoder(Model):
    def __init__(self, output_units, code_size=2):
        super().__init__()
        self.encoder = Sequential([
            Dense(16, activation='relu'),
            Dropout(0.1),
            Dense(8, activation='relu'),
            Dropout(0.1),
            Dense(4, activation='relu'),
            Dropout(0.1),
            Dense(code_size, activation='relu')
        ])
        self.decoder = Sequential([
              Dense(4, activation='relu'),
              Dropout(0.1),
              Dense(8, activation='relu'),
              Dropout(0.1),
              Dense(16, activation='relu'),
              Dropout(0.1),
              Dense(output_units, activation='sigmoid')
        ])
        
    def call(self, inputs):
        encoded = self.encoder(inputs)
        decoded = self.decoder(encoded)
        return decoded

In [22]:
model = AutoEncoder(output_units=x_train_scaled.shape[1])
model.compile(loss='msle', metrics=['mse'], optimizer='adam')
checkpointer = ModelCheckpoint(filepath='./model/keras_best.h5', verbos=1, save_best_only=True, save_weights_only=True, monitor='val_mse', mode='min')
earlystopping = EarlyStopping(monitor='val_mse', mode='min', verbose=1, patience=200)

## 5. 모델 학습

In [23]:
history = model.fit(x_train_scaled, x_train_scaled, epochs=400, batch_size=512, 
                    validation_split=0.2, callbacks=[checkpointer, earlystopping], workers=4)

Epoch 1/400
178/178 [==============================] - 3s 13ms/step - loss: 0.0105 - mse: 0.0227 - val_loss: 0.0043 - val_mse: 0.0107
Epoch 2/400
178/178 [==============================] - 2s 12ms/step - loss: 0.0029 - mse: 0.0060 - val_loss: 0.0039 - val_mse: 0.0101
Epoch 3/400
178/178 [==============================] - 2s 12ms/step - loss: 0.0024 - mse: 0.0050 - val_loss: 0.0039 - val_mse: 0.0100
Epoch 4/400
178/178 [==============================] - 2s 12ms/step - loss: 0.0023 - mse: 0.0048 - val_loss: 0.0039 - val_mse: 0.0101
Epoch 5/400
178/178 [==============================] - 2s 12ms/step - loss: 0.0022 - mse: 0.0046 - val_loss: 0.0039 - val_mse: 0.0100
Epoch 6/400
178/178 [==============================] - 2s 12ms/step - loss: 0.0022 - mse: 0.0046 - val_loss: 0.0039 - val_mse: 0.0101
Epoch 7/400
178/178 [==============================] - 2s 12ms/step - loss: 0.0021 - mse: 0.0045 - val_loss: 0.0035 - val_mse: 0.0090
Epoch 8/400
178/178 [==============================] - 2s 12ms

Epoch 62/400
178/178 [==============================] - 2s 12ms/step - loss: 0.0012 - mse: 0.0026 - val_loss: 0.0012 - val_mse: 0.0028
Epoch 63/400
178/178 [==============================] - 2s 12ms/step - loss: 0.0012 - mse: 0.0026 - val_loss: 0.0012 - val_mse: 0.0028
Epoch 64/400
178/178 [==============================] - 2s 12ms/step - loss: 0.0012 - mse: 0.0026 - val_loss: 0.0013 - val_mse: 0.0028
Epoch 65/400
178/178 [==============================] - 2s 12ms/step - loss: 0.0012 - mse: 0.0026 - val_loss: 0.0013 - val_mse: 0.0028
Epoch 66/400
178/178 [==============================] - 2s 12ms/step - loss: 0.0012 - mse: 0.0026 - val_loss: 0.0012 - val_mse: 0.0028
Epoch 67/400
178/178 [==============================] - 2s 12ms/step - loss: 0.0012 - mse: 0.0026 - val_loss: 0.0013 - val_mse: 0.0028
Epoch 68/400
178/178 [==============================] - 2s 12ms/step - loss: 0.0012 - mse: 0.0026 - val_loss: 0.0012 - val_mse: 0.0027
Epoch 69/400
178/178 [==============================] -

178/178 [==============================] - 2s 12ms/step - loss: 0.0012 - mse: 0.0025 - val_loss: 0.0013 - val_mse: 0.0028
Epoch 123/400
178/178 [==============================] - 2s 12ms/step - loss: 0.0012 - mse: 0.0025 - val_loss: 0.0013 - val_mse: 0.0028
Epoch 124/400
178/178 [==============================] - 2s 12ms/step - loss: 0.0012 - mse: 0.0025 - val_loss: 0.0013 - val_mse: 0.0029
Epoch 125/400
178/178 [==============================] - 2s 12ms/step - loss: 0.0012 - mse: 0.0025 - val_loss: 0.0012 - val_mse: 0.0028
Epoch 126/400
178/178 [==============================] - 2s 12ms/step - loss: 0.0012 - mse: 0.0025 - val_loss: 0.0012 - val_mse: 0.0027
Epoch 127/400
178/178 [==============================] - 2s 13ms/step - loss: 0.0012 - mse: 0.0025 - val_loss: 0.0013 - val_mse: 0.0029
Epoch 128/400
178/178 [==============================] - 2s 12ms/step - loss: 0.0012 - mse: 0.0025 - val_loss: 0.0013 - val_mse: 0.0029
Epoch 129/400
178/178 [==============================] - 2s 12

178/178 [==============================] - 2s 12ms/step - loss: 0.0012 - mse: 0.0025 - val_loss: 0.0012 - val_mse: 0.0028
Epoch 183/400
178/178 [==============================] - 2s 12ms/step - loss: 0.0012 - mse: 0.0025 - val_loss: 0.0013 - val_mse: 0.0029
Epoch 184/400
178/178 [==============================] - 2s 12ms/step - loss: 0.0012 - mse: 0.0025 - val_loss: 0.0013 - val_mse: 0.0030
Epoch 185/400
178/178 [==============================] - 2s 12ms/step - loss: 0.0012 - mse: 0.0025 - val_loss: 0.0013 - val_mse: 0.0029
Epoch 186/400
178/178 [==============================] - 2s 12ms/step - loss: 0.0012 - mse: 0.0025 - val_loss: 0.0013 - val_mse: 0.0028
Epoch 187/400
178/178 [==============================] - 2s 12ms/step - loss: 0.0012 - mse: 0.0025 - val_loss: 0.0013 - val_mse: 0.0029
Epoch 188/400
178/178 [==============================] - 2s 12ms/step - loss: 0.0012 - mse: 0.0025 - val_loss: 0.0012 - val_mse: 0.0028
Epoch 189/400
178/178 [==============================] - 2s 12

In [24]:
def find_threshold(model, x_train_scaled):
    reconstructions = model.predict(x_train_scaled)
    # provides losses of individual instances
    reconstruction_errors = tf.keras.losses.msle(reconstructions, x_train_scaled)
    # threshold for anomaly scores
    threshold = np.mean(reconstruction_errors.numpy()) + np.std(reconstruction_errors.numpy())
    return threshold

In [30]:
def get_predictions(model, x_val_scaled, threshold):
    predictions = model.predict(x_val_scaled)
    # provides losses of individual instances
    errors = tf.keras.losses.msle(predictions, x_val_scaled)
    # 0 = anomaly, 1 = normal
    anomaly_mask = pd.Series(errors) > threshold
    preds = anomaly_mask.map(lambda x: 1.0 if x == True else 0.0)
    return preds

In [26]:
threshold = find_threshold(model, x_train_scaled)
print(f"Threshold: {threshold}")

Threshold: 0.0022835944358149717


In [38]:
predictions = get_predictions(model, x_val_scaled, threshold)
accuracy_score(predictions, y_val_features)

0.9375658773100977

In [41]:
f1_score(predictions, y_val_features, average='macro')

0.4975575295036959